In [1]:
import os
import pathlib

import numpy as np
import os
import six.moves.urllib as urllib
import sys
import tarfile
import tensorflow as tf
import zipfile

from collections import defaultdict
from io import StringIO
from matplotlib import pyplot as plt
from PIL import Image
from IPython.display import display
from PIL import Image, ImageFont, ImageDraw, ImageEnhance

Import the object detection module.

In [2]:
from object_detection.utils import ops as utils_ops
from object_detection.utils import label_map_util
from object_detection.utils import visualization_utils as vis_util

import cv2
import time
import csv

import linecache
from scipy.spatial import distance

In [3]:
# patch tf1 into `utils.ops`
utils_ops.tf = tf.compat.v1

# Patch the location of gfile
tf.gfile = tf.io.gfile

# Model preparation 

In [4]:
class ModelLoader:
    def load_model(self, model_name):
        #If we want to download a new model
        #base_url = 'http://download.tensorflow.org/models/object_detection/'
        model_file = model_name + '.tar.gz'
        #model_dir = tf.keras.utils.get_file(fname=model_name, origin=base_url + model_file,untar=True)
        model_dir = "saved_models"/pathlib.Path(model_name)/"saved_model"

        model = tf.saved_model.load(str(model_dir))
        model = model.signatures['serving_default']

        return model

In [5]:
# List of the strings that is used to add correct label for each box.

PATH_TO_LABELS = 'F:/Machine_learning/Online-study/computer-vision/person_detection_work/models/research/object_detection/data/mscoco_label_map.pbtxt'
category_index = label_map_util.create_category_index_from_labelmap(PATH_TO_LABELS, use_display_name=True)

# Detection

#### Loading the saved object detection model

In [6]:
model_name = 'ssd_mobilenet_v2_coco_2018_03_29'
detection_model = ModelLoader().load_model(model_name)

INFO:tensorflow:Saver not created because there are no variables in the graph to restore


Add a wrapper function to call the model, and cleanup the outputs:

In [7]:
class ObjectDetection:
    def run_inference_for_single_image(self, model, image):
        image = np.asarray(image)
        # The input needs to be a tensor, convert it using `tf.convert_to_tensor`.
        input_tensor = tf.convert_to_tensor(image)
        # The model expects a batch of images, so add an axis with `tf.newaxis`.
        input_tensor = input_tensor[tf.newaxis,...]
        # Run inference
        output_dict = model(input_tensor)
        num_detections = int(output_dict.pop('num_detections'))
        output_dict = {key:value[0, :num_detections].numpy() 
                     for key,value in output_dict.items()}
        output_dict['num_detections'] = num_detections

        # detection_classes should be ints.
        output_dict['detection_classes'] = output_dict['detection_classes'].astype(np.int64)
        # Handle models with masks:

        if 'detection_masks' in output_dict:
        # Reframe the the bbox mask to the image size.
            detection_masks_reframed = utils_ops.reframe_box_masks_to_image_masks(
                      output_dict['detection_masks'], output_dict['detection_boxes'],
                       image.shape[0], image.shape[1])
            detection_masks_reframed = tf.cast(detection_masks_reframed > 0.5,
                                               tf.uint8)
            output_dict['detection_masks_reframed'] = detection_masks_reframed.numpy()
        return output_dict
    
    def show_inference(self, model, image_path):
        # the array based representation of the image will be used later in order to prepare the
        # result image with boxes and labels on it.
        image_np = image_path
        # Actual detection.
        output_dict = self.run_inference_for_single_image(model, image_np)
        # Visualization of the results of a detection.
        vis_util.visualize_boxes_and_labels_on_image_array(
          image_np,
          output_dict['detection_boxes'],
          output_dict['detection_classes'],
          output_dict['detection_scores'],
          category_index,
          instance_masks=output_dict.get('detection_masks_reframed', None),
          use_normalized_coordinates=True,
          line_thickness=8)

        return image_np, output_dict

Run it on each test image and show the results:

In [8]:
class GetClassAndBoundingBox:
    def get_boxes(self, op_dict):
        boxes = []
        classes = []
        for i in range(op_dict["num_detections"]):
            if op_dict["detection_scores"][i] > 0.5:
                boxes.append(op_dict["detection_boxes"][i])
                classes.append(op_dict["detection_classes"][i])
        return boxes, classes

In [9]:
class CalculateCentroids:
    def get_x_y_center(self, box, im_width, im_height):
        ymin, xmin, ymax, xmax = box
        (left, right, top, bottom) = (xmin * im_width, xmax * im_width, ymin * im_height, ymax * im_height)

        # Calculate height and width of the bounding box
        height = bottom - top
        width = right - left
        # Calculating the centroids of the bounding box's X and Y axis

        half_height = height/2
        half_width = width/2
        centroid_of_x_axis = top+half_height 
        centroid_of_y_axis = left+half_width
        return centroid_of_x_axis, centroid_of_y_axis, left, right, top, bottom

In [10]:
class UniqueIdentification:
    
    def check_for_id_fluctualtion(self, id_details, key_val, new_ids, frame):
        #print("Before Adding a new id we have to check for ID fluctuation................")
        distances_from_previous_idd_centroids = []
        least_distance = 100000000000000
        index = 0
        for id_d in id_details:
            previous_centroids = id_d["previous_centroid"].split(',')
            last_seen_frame = id_d["last_seen_on_frame"]
            frame_difference = frame - last_seen_frame
            prv_c1 = float(previous_centroids[0])
            prv_c2 = float(previous_centroids[1])
            current_centroid = key_val.split(",")
            crnt_c1 = float(current_centroid[0])
            crnt_c2 = float(current_centroid[1])
            d = distance.euclidean((prv_c1, prv_c2),(crnt_c1,crnt_c2))
            if d<least_distance and frame_difference < 10:
                least_distance = d
                temp_assigned_id = id_d["id"]
                index_of_assigned_id = index
            index += 1
        if least_distance > 150:
            return False ,new_ids, id_details

        new_ids[key_val] = temp_assigned_id
        return True, new_ids, id_details
    
    def update_id_details(self, id_details, new_ids, frame):
        for centroid, id_ in new_ids.items():
            id_details[id_]["previous_centroid"] = centroid
            id_details[id_]["last_seen_on_frame"] = frame
        return id_details
    
    def register_new_id(self, distance_to_all_points, new_ids, last_used_id, centroids, id_details, frame):
        """
        If the centroid is having distancing from all the points greter than some threshold,
        then this function will add new ID to that bounding box.
        """
        all_points = len(distance_to_all_points)
        counter_for_adding_new_id = 0
        index = 0
        for prev_dist in distance_to_all_points:
            if prev_dist > 150:
                counter_for_adding_new_id += 1
            index += 1
        if all_points == counter_for_adding_new_id:
            fount_fluctuation, new_ids, id_details = self.check_for_id_fluctualtion(id_details, centroids, new_ids, frame)
            if not fount_fluctuation:
                new_ids[centroids] = last_used_id
                id_details.append({'id': last_used_id, 'last_seen_on_frame': frame, 'previous_centroid': centroids})
                last_used_id += 1

        return new_ids, last_used_id, id_details
    
    def find_least_distance(self, centroid, id_, box, distance_to_all_points, least_distance, temp_distance_keeper, new_ids, tmp_assigned_id):
        '''
        This function is responsible for finding nearest previous centroid of the current bounding box's centroid.
        Taks input as 
        centroid                 | string : centroids of the previous bounding boxes
        id_                      | int : id assigned to previous centroid
        box                      | tuple : location of current bounding box
        distance_to_all_points   | list of distance calculated to all the previous centroids
        least_distance           | int: least distance among previously calculated distances with all the previous centroids for the current bounding box
        temp_distance_keeper     | list of dictionalry which holds id, distance, current_centroids, previous_centroid 
                                 | (This will be helpful when we will have more that one points having assigned same id)
        new_ids                  | List of assigned ids for each bounding boxes till now.
        '''
        try:
            centroids = centroid.split(',')
            pc1 = float(centroids[0])
            pc2 = float(centroids[1])
            d = distance.euclidean((pc1, pc2),(box[0],box[1]))
            distance_to_all_points.append(d)
            if d<least_distance:
                least_distance = d
                tmp_assigned_id = id_
                key_val = str(box[0]) + "," + str(box[1])
                same_id_found = 0
                for tmp_dist in temp_distance_keeper:
                    '''
                    This checks if id is already assigned before, if id is assigned then check the distance of the,
                    2 centroids and chooses the point which have least distance.
                    '''
                    if tmp_assigned_id == tmp_dist["id"]:
                        same_id_found = 1
                        if least_distance < tmp_dist["distance"]:
                            tmp_dist["distance"] = least_distance
                            tmp_dist["current_point"] = key_val
                            new_ids[key_val] = tmp_dist["id"]
                if not same_id_found:
                    temp_distance_keeper.append({"id":tmp_assigned_id, "distance":least_distance, "current_point":key_val, " previous_point ":centroid})
                    new_ids[key_val] = tmp_assigned_id
            return new_ids, temp_distance_keeper, distance_to_all_points, least_distance, tmp_assigned_id
        except:
            exc_type, exc_obj, tb = sys.exc_info()
            f = tb.tb_frame
            lineno = tb.tb_lineno
            print('find_least_distance EXCEPTION IN LINE ', lineno , " exception : " , exc_obj)
            return {}, [], distance_to_all_points, 1000000000000000, None
        
    def update_ball_centroid(self, previous_frame_ball_ids, box, frame):
        try:
            new_ball_centroid = {}
            ball_tmp_id = previous_frame_ball_ids["id"]
            centroid = str(box[0]) + "," + str(box[1])
            new_ball_centroid[centroid] = ball_tmp_id
            return new_ball_centroid
        except:
            exc_type, exc_obj, tb = sys.exc_info()
            f = tb.tb_frame
            lineno = tb.tb_lineno
            print('update_ball_centroid EXCEPTION IN LINE ', lineno , " exception : " , exc_obj)
            return {}, [], distance_to_all_points, 1000000000000000, None

    def find_nearest_point(self, boxes, classes, previous_centroids, previous_ball_centroid, last_used_id, im_width, im_height, image_pil, font, id_details, frame):
        """
        boxes:   bounding box location for objects detected in current frame
        classes: class of the detected object
        previous_centroid: centroids and Ids of last frame for detecting persons
        previous_ball_centroid: centroids and Ids of last frame for detecting ball
        """
        try:
            Ids = previous_centroids["id"]

            previous_ids = previous_centroids["id"].values()
            temp_distance_keeper = []
            new_ids = {}
            ball_id_list = {}

            for box, cls in zip(boxes, classes):
                centroid_of_x_axis, centroid_of_y_axis, left, right, top, bottom = CalculateCentroids().get_x_y_center(box, im_width, im_height)
                box = (centroid_of_x_axis, centroid_of_y_axis)

                height = bottom - top
                width = right - left

                distance_to_all_points = []
                distance_to_all_ball_points = []

                least_distance = 100000000000000

                tmp_assigned_id = None
                tmp_ball_assigned_id = None
                key_val = str(box[0]) + "," + str(box[1])
                if cls == 1:
                    for centroid, id_ in Ids.items():
                        new_ids, temp_distance_keeper, distance_to_all_points, least_distance, tmp_assigned_id = self.find_least_distance(centroid, id_, box, distance_to_all_points, least_distance, temp_distance_keeper, new_ids, tmp_assigned_id)
                    new_ids, last_used_id, id_details = self.register_new_id(distance_to_all_points, new_ids, last_used_id, key_val, id_details, frame)
                    id_details = self.update_id_details(id_details, new_ids, frame)

                elif cls == 40 or cls == 37:
                    if not previous_ball_centroid:
                        ball_id_list = FirstFrameIdentifications().first_frame_for_ball(box, im_width, im_height, frame)
                        tmp_assigned_id = list(ball_id_list.values())[0]
                    else:
                        print("FOUND THE BALL IN THE FRAME................................")
                        ball_id_list = self.update_ball_centroid(previous_ball_centroid, box,  frame)
                        tmp_assigned_id = list(ball_id_list.values())[0]

                try:
                    tmp_assigned_id = new_ids[key_val]
                except:
                    pass

                draw = ImageDraw.Draw(image_pil)
                draw.ellipse((centroid_of_y_axis, centroid_of_x_axis, centroid_of_y_axis+10, centroid_of_x_axis+10), fill = 'yellow',outline="yellow")

                draw.rectangle([(centroid_of_y_axis-20, centroid_of_x_axis), (centroid_of_y_axis+50, centroid_of_x_axis+30)], width=1, fill=(255, 0, 0))
                draw.text((centroid_of_y_axis-20, centroid_of_x_axis), "Id: " + str(tmp_assigned_id),fill=(0, 255, 255),font=font,align="center")

            return new_ids, last_used_id, image_pil, id_details, ball_id_list
        except:
            exc_type, exc_obj, tb = sys.exc_info()
            f = tb.tb_frame
            lineno = tb.tb_lineno
            filename = f.f_code.co_filename
            linecache.checkcache(filename)
            line = linecache.getline(filename, lineno, f.f_globals)
            print('find_nearest_point EXCEPTION IN LINE ', lineno , " exception : " , exc_obj)
            return 0, 0, 0
        
    

In [11]:
class FirstFrameIdentifications:
    def first_frame_for_ball(self, box, im_width, im_height, frame):
        try:
            ball_tmp_id = 0
            ball_id_list = {}

            centroid = str(box[0]) + "," + str(box[1])
            ball_id_list[centroid] = "B" + str(ball_tmp_id)
            ball_tmp_id += 1
            return ball_id_list
        except:
            exc_obj = sys.exc_info()[2]
            f = tb.tb_frame
            lineno = tb.tb_lineno
            print('first_frame_for_ball EXCEPTION IN LINE ', lineno , " exception : " , exc_obj)
            return 0, 0, 0
    
    def for_first_frame(self, boxes, classes, im_width, im_height, image_pil, font, id_details, ball_id_details, frame):
        '''
        This function assigns ids to all detected number of bounding boxes found in first frame of the video.
        takes input of location of the bounding box.
        returns dictionary in which key is centroid of the bounding box and value is assigned id.
        '''

        temp_id = 0
        ball_tmp_id = 0
        id_list = {}
        ball_id_list = {}
        for box, cls in zip(boxes, classes):
            centroid_of_x_axis, centroid_of_y_axis, left, right, top, bottom = CalculateCentroids().get_x_y_center(box, im_width, im_height)
            box = (centroid_of_x_axis, centroid_of_y_axis)

            height = bottom - top
            width = right - left
            centroid = str(box[0]) + "," + str(box[1])
            if cls == 1:
                id_list[centroid] = temp_id
                id_details.append({"id":temp_id, "last_seen_on_frame": frame, "previous_centroid": centroid})
                draw = ImageDraw.Draw(image_pil)
                draw.ellipse((centroid_of_y_axis, centroid_of_x_axis, centroid_of_y_axis+10, centroid_of_x_axis+10), fill = 'yellow',outline="yellow")

                draw.rectangle([(centroid_of_y_axis-20, centroid_of_x_axis), (centroid_of_y_axis+50, centroid_of_x_axis+30)], width=1, fill=(255, 0, 0))
                draw.text((centroid_of_y_axis-20, centroid_of_x_axis), "Id: " + str(temp_id),fill=(0, 255, 255),font=font,align="center")
                temp_id += 1
            elif cls == 40 or cls == 37:
                ball_id_list[centroid] = ball_tmp_id
                ball_id_details.append({"id":"B" + str(ball_tmp_id), "last_seen_on_frame": frame, "previous_centroid": centroid})
                draw = ImageDraw.Draw(image_pil)
                draw.ellipse((centroid_of_y_axis, centroid_of_x_axis, centroid_of_y_axis+10, centroid_of_x_axis+10), fill = 'yellow',outline="yellow")

                draw.rectangle([(centroid_of_y_axis-20, centroid_of_x_axis), (centroid_of_y_axis+50, centroid_of_x_axis+30)], width=1, fill=(255, 0, 0))
                draw.text((centroid_of_y_axis-20, centroid_of_x_axis), "Id: " + "B" + str(ball_tmp_id),fill=(0, 255, 255),font=font,align="center")
                ball_tmp_id += 1

        return id_list, temp_id, image_pil, id_details, ball_id_list, ball_id_details, ball_tmp_id

In [31]:
class CheckBallPossession:
    def find_nearest_human_to_ball(self, player_ids, ball_id):
        ball_centroid = ball_id["previous_centroid"].split(",")
        player_centroids = player_ids["id"]
        least_distance = 100000
        for player_centroid, assigned_id in player_centroids.items():
            pc1 = player_centroid.split(",")[0]
            pc2 = player_centroid.split(",")[1]

            d = distance.euclidean((float(pc1), float(pc2)),(float(ball_centroid[0]), float(ball_centroid[1])))
            if d<least_distance:
                least_distance = d
                nearest_player_id = assigned_id
        return nearest_player_id, least_distance
    
    def check_ball_possession(self, distance, seek_time, 
                          in_the_air, ball_occupied_in_progress,  
                          ball_occupation_summary, in_air_time, 
                          temp_ball_occupier = None):
        try:
            if distance < 150 and in_air_time and not ball_occupation_summary:
                ball_occupied_in_progress = True
                in_the_air = False
                ball_occupation_summary = {"ball_position": "possessed", "from": seek_time, "to":None,"player_id":temp_ball_occupier}
                in_air_time["to"] = seek_time
                iteration_completed = "catched"
                return ball_occupied_in_progress, in_the_air, ball_occupation_summary, in_air_time, iteration_completed

            elif distance < 150 and not ball_occupation_summary:
                ball_occupied_in_progress = True
                in_the_air = False
                ball_occupation_summary = {"ball_position":"possessed", "from":seek_time, "to":None,"player_id":temp_ball_occupier}
                in_air_time = {}    
                iteration_completed = False
                return ball_occupied_in_progress, in_the_air, ball_occupation_summary, in_air_time, iteration_completed

            elif distance < 150 and in_the_air:
                ball_occupied_in_progress = True
                in_the_air = False
                ball_occupation_summary = {"ball_position": "possessed", "from": seek_time, "to":None,"player_id":temp_ball_occupier}
                in_air_time["to"] = seek_time
                iteration_completed = "catched"
                return ball_occupied_in_progress, in_the_air, ball_occupation_summary, in_air_time, iteration_completed

            elif distance >= 150 and ball_occupied_in_progress and not in_the_air:
                ball_occupied_in_progress = False
                in_the_air = True
                ball_occupation_summary["to"] = seek_time
                in_air_time = {"ball_position":"in_air", "from":seek_time, "to":None,"player_id":"None"}
                iteration_completed = "thrown"
                return ball_occupied_in_progress, in_the_air, ball_occupation_summary, in_air_time, iteration_completed

            elif distance >= 150 and in_the_air:
                iteration_completed = False
                return ball_occupied_in_progress, in_the_air, ball_occupation_summary, in_air_time, iteration_completed

            elif distance >= 150 and not in_air_time:
                ball_occupied_in_progress = False
                in_the_air = True
                ball_occupation_summary = {}
                in_air_time = {"ball_position":"in_air", "from":seek_time, "to":None,"player_id":"None"}
                iteration_completed = False
                return ball_occupied_in_progress, in_the_air, ball_occupation_summary, in_air_time, iteration_completed
            else:
                iteration_completed = False
                return ball_occupied_in_progress, in_the_air, ball_occupation_summary, in_air_time, iteration_completed
        except Exception as e:
            print(e)
            return ball_occupied_in_progress, in_the_air, ball_occupation_summary, in_air_time, iteration_completed

In [32]:
class BallPossession:
    def ball_detected_first_time(self, ball_possession, Ids, ballIds, seek_time, in_the_air, ball_occupied_in_progress,  ball_occupation_summary, in_air_time, temp_ball_occupier):
        try:
            ball_detected = 1
            nearest_player_id, least_distance = CheckBallPossession().find_nearest_human_to_ball(Ids, ballIds)
            temp_ball_occupier = nearest_player_id
            temp_from_seek_time = seek_time
            ball_occupation_summary, in_air_time = {}, {}
            ball_occupied_in_progress, in_the_air, ball_occupation_summary, in_air_time, iteration_completed = CheckBallPossession().check_ball_possession(least_distance, seek_time, in_the_air, ball_occupied_in_progress,  ball_occupation_summary, in_air_time, temp_ball_occupier)                
            if iteration_completed == "catched":
                ball_possession.append(in_air_time)
                in_air_time = {}
            elif iteration_completed == "thrown":
                ball_possession.append(ball_occupation_summary)
                ball_occupation_summary = {}
            return ball_possession, ball_occupation_summary, in_air_time, in_the_air, ball_occupied_in_progress
        except Exception as e:
            exc_type, exc_obj, tb = sys.exc_info()
            f = tb.tb_frame
            lineno = tb.tb_lineno
            print('EXCEPTION IN LINE ', lineno , " exception : " , exc_obj)
    def ball_detection_in_process(self, ball_possession, Ids, ballIds, seek_time, in_the_air, ball_occupied_in_progress,  ball_occupation_summary, in_air_time, temp_ball_occupier):
        try:
            nearest_player_id, least_distance = CheckBallPossession().find_nearest_human_to_ball(Ids, ballIds)
            temp_ball_occupier = nearest_player_id
            ball_occupied_in_progress, in_the_air, ball_occupation_summary, in_air_time, iteration_completed = CheckBallPossession().check_ball_possession(least_distance, seek_time, in_the_air, ball_occupied_in_progress,  ball_occupation_summary, in_air_time, temp_ball_occupier)
            if iteration_completed == "catched":
                ball_possession.append(in_air_time)
                in_air_time = {}
            elif iteration_completed == "thrown":
                ball_possession.append(ball_occupation_summary)
                ball_occupation_summary = {}
            return ball_possession, ball_occupation_summary, in_air_time, in_the_air, ball_occupied_in_progress
        except Exception as e:
            exc_type, exc_obj, tb = sys.exc_info()
            f = tb.tb_frame
            lineno = tb.tb_lineno
            print('EXCEPTION IN LINE ', lineno , " exception : " , exc_obj)

In [33]:
import datetime

In [34]:
vid = "F:/Machine_learning/WORK/video/ball-possesion5.mp4"

In [36]:
cap = cv2.VideoCapture(vid)

# For calculatig seek time
fps = cap.get(cv2.CAP_PROP_FPS)      
frame_count = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
print(frame_count)
duration = frame_count/fps

id_details = []
ball_id_details = []

Ids = []
ballIds = {}

previous_id = 0
previous_ball_id = 0

ball_detected = 0

ball_possession = []

temp_from_seek_time = None
temp_ball_occupier = None

ball_occupied_in_progress = None
in_the_air = None



font = ImageFont.truetype('arial.ttf', 30)
while(cap.isOpened()):
    #success is boolean and image contains frame of the video
    try:
        success, vimg = cap.read()
        if success:
            img, op_dict = ObjectDetection().show_inference(detection_model, vimg)
            frame_count = int(cap.get(cv2.CAP_PROP_POS_FRAMES))
            
            print("------------",frame_count,"--------------")
            seek_time = cap.get(cv2.CAP_PROP_POS_MSEC)/1000
            seek_time = round(seek_time, 2)
            
            if seek_time >= 4.80:
                print("")
                break

            # Fetching the bounding box coordinates
            boxes, classes = GetClassAndBoundingBox().get_boxes(op_dict)

            image_pil = Image.fromarray(np.uint8(img)).convert('RGB')
            im_width, im_height = image_pil.size
            if frame_count == 1:
                id_list, previous_id, image_pil, id_details, ball_id_list, ball_id_details, previous_ball_id = FirstFrameIdentifications().for_first_frame(boxes, classes, im_width, im_height, image_pil, font, id_details, ball_id_details, frame_count)
                Ids.append({"frame":frame_count, "id":id_list})
                ballIds["frame"] = frame_count
                ballIds = ball_id_list
            else:
                id_list, previous_id, image_pil, id_details, ball_id_list = UniqueIdentification().find_nearest_point(boxes, classes, Ids[frame_count-2], ballIds, previous_id, im_width, im_height, image_pil, font, id_details, frame_count)
                Ids.append({"frame":frame_count, "id":id_list})
                if ball_id_list:
                    ballIds["frame"] = frame_count
                    ballIds["id"] = list(ball_id_list.values())[0]
                    ballIds["previous_centroid"] = list(ball_id_list.keys())[0]
            
            if ((1 in classes) and ((37 in classes) or (40 in classes))) and not ball_detected:
                ball_detected = 1
                nearest_player_id, least_distance = CheckBallPossession().find_nearest_human_to_ball(Ids[frame_count-1], ballIds)
                print("Nearest player is ", nearest_player_id, " and distance to that player is ", least_distance)
                temp_ball_occupier = nearest_player_id
                temp_from_seek_time = seek_time
                ball_occupation_summary, in_air_time = {}, {}
                ball_occupied_in_progress, in_the_air, ball_occupation_summary, in_air_time, iteration_completed = CheckBallPossession().check_ball_possession(least_distance, seek_time, in_the_air, ball_occupied_in_progress,  ball_occupation_summary, in_air_time, temp_ball_occupier)                
                if iteration_completed == "catched":
                    ball_possession.append(in_air_time)
                    in_air_time = {}
                elif iteration_completed == "thrown":
                    ball_possession.append(ball_occupation_summary)
                    ball_occupation_summary = {}
                    
            elif ball_detected and 1 in classes:
                nearest_player_id, least_distance = CheckBallPossession().find_nearest_human_to_ball(Ids[frame_count-1], ballIds)
                print("Nearest player is ", nearest_player_id, " and distance to that player is ", least_distance)
                temp_ball_occupier = nearest_player_id
                ball_occupied_in_progress, in_the_air, ball_occupation_summary, in_air_time, iteration_completed = CheckBallPossession().check_ball_possession(least_distance, seek_time, in_the_air, ball_occupied_in_progress,  ball_occupation_summary, in_air_time, temp_ball_occupier)
                if iteration_completed == "catched":
                    ball_possession.append(in_air_time)
                    in_air_time = {}
                elif iteration_completed == "thrown":
                    ball_possession.append(ball_occupation_summary)
                    ball_occupation_summary = {}
                    
            elif (ball_detected and (1 in classes) and (37 in classes or 40 in classes)):
                nearest_player_id, least_distance = CheckBallPossession().find_nearest_human_to_ball(Ids[frame_count-1], ballIds)
                print("Nearest player is ", nearest_player_id, " and distance to that player is ", least_distance)
                temp_ball_occupier = nearest_player_id
                ball_occupied_in_progress, in_the_air, ball_occupation_summary, in_air_time, iteration_completed = CheckBallPossession().check_ball_possession(least_distance, seek_time, in_the_air, ball_occupied_in_progress,  ball_occupation_summary, in_air_time, temp_ball_occupier)
                if iteration_completed == "catched":
                    ball_possession.append(in_air_time)
                    in_air_time = {}
                elif iteration_completed == "thrown":
                    ball_possession.append(ball_occupation_summary)
                    ball_occupation_summary = {}

            np.copyto(img, np.array(image_pil))
            vimg = cv2.resize(img, (1200,690))
            vimg = cv2.putText(vimg,"Time : " + str(seek_time), (20,50), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 0, 255), 3)
            if frame_count == frame_count - 1:
                vimg = cv2.putText(vimg,"Inferences written in csv file.", (50,50), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 0, 255), 2)
            cv2.imshow("Frame",vimg)
            #time.sleep(1)
            key = cv2.waitKey(1)
            if key == 27:
                break
        else:
            break
    except Exception as e:
        exc_type, exc_obj, tb = sys.exc_info()
        f = tb.tb_frame
        lineno = tb.tb_lineno
        print('EXCEPTION IN LINE ', lineno , " exception : " , exc_obj)
        cap.release()
        cv2.destroyAllWindows()
if ball_possession:
    date = str(datetime.datetime.today()).split(" ")[0]
    file_name = os.path.join("inferences","inference" + date + ".csv")
    with open(file_name, 'w', newline='')  as output_file:
        dict_writer = csv.DictWriter(output_file, ball_possession[0].keys())
        dict_writer.writeheader()
        dict_writer.writerows(ball_possession)

cap.release()
cv2.destroyAllWindows()


175
------------ 1 --------------
------------ 2 --------------
------------ 3 --------------
------------ 4 --------------
------------ 5 --------------
------------ 6 --------------
------------ 7 --------------
------------ 8 --------------
------------ 9 --------------
------------ 10 --------------
------------ 11 --------------
------------ 12 --------------
------------ 13 --------------
------------ 14 --------------
------------ 15 --------------
------------ 16 --------------
------------ 17 --------------
------------ 18 --------------
------------ 19 --------------
------------ 20 --------------
------------ 21 --------------
------------ 22 --------------
------------ 23 --------------
------------ 24 --------------
Nearest player is  2  and distance to that player is  125.80438252619462
------------ 25 --------------
Nearest player is  2  and distance to that player is  89.92438852677445
------------ 26 --------------
Nearest player is  2  and distance to that player is  

------------ 91 --------------
Nearest player is  18  and distance to that player is  447.78514974887196
------------ 92 --------------
Nearest player is  18  and distance to that player is  412.0220563839034
------------ 93 --------------
Nearest player is  18  and distance to that player is  333.07364445858803
------------ 94 --------------
Nearest player is  18  and distance to that player is  246.74095185557294
------------ 95 --------------
Nearest player is  18  and distance to that player is  197.60426655598042
------------ 96 --------------
Nearest player is  18  and distance to that player is  208.9474781543081
------------ 97 --------------
Nearest player is  18  and distance to that player is  279.6951774007297
------------ 98 --------------
Nearest player is  18  and distance to that player is  367.57055133091734
------------ 99 --------------
Nearest player is  22  and distance to that player is  206.4945657268455
------------ 100 --------------
Nearest player is  22  and 

In [21]:
print(str(datetime.datetime.today()).split(" ")[0])

2020-08-14


In [22]:
ball_possession

[{'ball_position': 'possessed', 'from': 0.92, 'to': 1.4, 'possessed_by': 2},
 {'ball_position': 'in_air', 'from': 1.4, 'to': 3.96},
 {'ball_position': 'possessed', 'from': 3.96, 'to': 4.36, 'possessed_by': 22},
 {'ball_position': 'in_air', 'from': 4.36, 'to': 4.72},
 {'ball_position': 'possessed', 'from': 4.72, 'to': 4.76, 'possessed_by': 22}]

### For image

In [24]:
img = cv2.imread("rugby.png")
img, op_dict = show_inference(detection_model, img)
boxes = []
min_threshold = 0.55
for i in range(op_dict["num_detections"]):
    if op_dict["detection_scores"][i] > min_threshold:
        boxes.append(op_dict["detection_boxes"][i])

image_pil = Image.fromarray(np.uint8(img)).convert('RGB')
im_width, im_height = image_pil.size

for box in boxes:
    ymin, xmin, ymax, xmax = box
    (left, right, top, bottom) = (xmin * im_width, xmax * im_width, ymin * im_height, ymax * im_height)
    
    height = bottom - top
    width = right - left
    half_height = height/2
    half_width = width/2
    centroid_of_x_axis = top+half_height 
    centroid_of_y_axis = left+half_width

    draw = ImageDraw.Draw(image_pil)
    draw.rectangle([(left, top), (right, bottom)], width=1)
    draw.ellipse((centroid_of_y_axis, centroid_of_x_axis, centroid_of_y_axis+10, centroid_of_x_axis+10), fill = 'yellow')
    
    np.copyto(img, np.array(image_pil))

image_pil.show()
cv2.imshow("Image",img)
cv2.waitKey(0)
cv2.destroyAllWindows()